<!-- dom:TITLE: Modeling the Worst: A Zombie outbreak -->
# Modeling the Worst: A Zombie outbreak
<!-- dom:AUTHOR: MOD510: Mandatory project \#3 -->
<!-- Author: -->  
**MOD510: Mandatory project \#3**  
<!-- dom:AUTHOR: Deadline: 10. November 2019 -->
<!-- Author: --> **Deadline: 10. November 2019**

Date: **Oct 25, 2019**

In [1]:
%matplotlib inline

import numpy as np
import scipy as sp
import datetime
import matplotlib.pyplot as plt
import pandas as pd

**Learning objectives.**
By completing this project, the student will:

* Learn how to escape from a zombie threat using wisdom from simple *compartment models*.

* Learn how one can model the spreading of diseases mathematically.

* Apply the model to study the outbreak of the Ebola virus in West-Africa in the period 2014-2015.

* Implement a selection of such models in Python, using standard methods.

* Gain intuition about which parameters are important by performing model sensitivity runs.


# Introduction
<div id="intro_compartment_models"></div>

In this project we are modeling the worst, but expecting (solving for) the
best. What might happen if zombies invade a little village in Norway, and
how much time do the inhabitants have to postpone or prevent the apocalypse?
After modeling zombies, we will apply the same kind of mathematical model to
real data for outbreaks of the Ebola virus in West Africa (2014-2016).

<!-- dom:FIGURE: [fig-project3/wkd.png, width=400 frac=1.0] Prepare for the worst (From the AMC show ''The Walking Dead'') <div id="fig:p3:wkd"></div> -->
<!-- begin figure -->
<div id="fig:p3:wkd"></div>

<p>Prepare for the worst (From the AMC show ''The Walking Dead'')</p>
<img src="fig-project3/wkd.png" width=400>

<!-- end figure -->


**<font color="white">Before we start:</font>.**

<font color="white">In this project, more than any other, you have to have control over numerical
uncertainty, and instabilities - your very future may depend on it!</font>



# Modeling a zombie outbreak - $SZ$ model

All of our mathematical models will be deterministic
*compartment models* [[kermack1927contribution]](#kermack1927contribution).
We let go of detailed descriptions, to gain equations that are mathematical
tractable. Assume for simplicity that we only have two
groups (compartments) of individuals:

1. $S$ - Susceptible: humans that are exposed to being turned into a zombie.

2. $Z$ - Zombies, humans that have been bitten or scratched by zombies.

<!-- dom:FIGURE: [fig-project3/ZI.png, width=400 frac=1.0] A two-compartment model. Note that only transport from the class of exposed humans to zombies are allowed. Zombies are *never* cured, if you are bitten you are dead, dead, dead, ... <div id="fig:human_zombies"></div> -->
<!-- begin figure -->
<div id="fig:human_zombies"></div>

<p>A two-compartment model. Note that only transport from the class of exposed humans to zombies are allowed. Zombies are <em>never</em> cured, if you are bitten you are dead, dead, dead, ...</p>
<img src="fig-project3/ZI.png" width=400>

<!-- end figure -->


The compartments are assumed to be *well mixed*, meaning that pairs of
individuals in the population interact with *equal probability*.
To develop a mathematical model, we need to estimate the rate of flow
between the two compartments in [figure](#fig:human_zombies).
During a time interval $\Delta{t}$, a certain number of human-zombie pairs
will meet, and in each such encounter there is a chance that the
human is infected. Let $\mathcal{C}(N)$ be the rate at which *any*
individual in the population contacts another individual, i.e., the average
number of contacts made per unit time. Since at time $t$ there are $Z(t)$
zombies, we assume the probability of meeting a zombie is $Z(t)/N$. Finally,
let $p_Z$ denote the probability that a human-zombie encounter results in
an infection. Then, we can model the reduction in the human population going
from time $t$ to $t+\Delta t$ as

<!-- Equation labels as ordinary links -->
<div id="eq:p3:pzi"></div>

$$
\begin{equation}
S(t+\Delta t)-S(t)=-\beta\cdot\Delta t\cdot\frac{Z(t)}{N}\cdot{S(t)}\,,
\label{eq:p3:pzi} \tag{1}
\end{equation}
$$

where we have defined $\beta=p_{Z}\cdot{\mathcal{C}(N)}$.

**<font color="white">Notice</font>.**

<font color="white">We have here assumed the contact-rate
to be a constant independent of population size. Other assumptions are
possible, and care should be taken when interpreting results from the
literature [[begon2002clarification]](#begon2002clarification).</font>



The $\beta$ factor implicitly contains a lot of biomedical, physical, and
sociological factors; for example, if the humans are unfit, they might not be able to
escape the zombies.
In the beginning of a zombie outbreak, $\beta$ is likely to be large, because
humanity might not understand the severity of the situation, or they may be
in denial. Thus, $\beta$ is most likely a time dependent parameter, which
will decrease once humans start to realize that zombies are dangerous.

The total population count $N=S(t)+Z(t)$ is regarded as constant in our simple
model, which means that we ignore phenomena such as birth, natural death,
immigration/emigration, etc.
Dividing both sides of the above equation with $\Delta t$, and taking the
limit $\Delta t\to 0$, we find:

<!-- Equation labels as ordinary links -->
<div id="eq:SIR1"></div>

$$
\begin{equation}
\label{eq:SIR1} \tag{2}
\frac{\mathrm{d}S(t)}{\mathrm{d}t}=-\beta\cdot\frac{S(t)Z(t)}{N}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:SIR2"></div>

$$
\begin{equation}  
\label{eq:SIR2} \tag{3}
\frac{\mathrm{d}Z(t)}{\mathrm{d}t}=\frac{\mathrm{d}(N-S)}{\mathrm{d}t}
= -\frac{\mathrm{d}S(t)}{\mathrm{d}t}
=\beta\cdot\frac{S(t)Z(t)}{N}\,.
\end{equation}
$$

Clearly, if there are no infected individuals at time zero, nothing will
happen later either. We shall therefore assume that
$Z_{0}\equiv{Z(0)}\geq{1}$; typically $Z_0=1$.



# Exercise 1: Analytical solution (OPTIONAL)
<div id="zombie_human_model"></div>

Show that the analytical solution to the SZ-model, equations
([2](#eq:SIR1)) and ([3](#eq:SIR2)), is

<!-- Equation labels as ordinary links -->
<div id="eq:SZ_analytical2"></div>

$$
\begin{equation}
\label{eq:SZ_analytical2} \tag{4}
S(t)=\frac{(S_0+Z_0)\frac{S_0}{Z_0}\exp(-\beta{t})}{1+\frac{S_0}{Z_0}\exp(-\beta{t})} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}  
Z(t)=\frac{S_0+Z_0}{1+\frac{S_0}{Z_0}\exp(-\beta{t})}
\,,
\label{_auto1} \tag{5}
\end{equation}
$$

where $S_0=S(0)$, and thus $S_0+Z_0=N$.


# Exercise 2: No hope?
<div id="no_hope"></div>

<!-- dom:FIGURE: [fig-project3/sd.png, width=400 frac=1.0] A view from Sokndal (left) and Dirdal (right). -->
<!-- begin figure -->

<p>A view from Sokndal (left) and Dirdal (right).</p>
<img src="fig-project3/sd.png" width=400>

<!-- end figure -->


Sokndal and Dirdal are two small villages in Rogaland. Two scientists from
these places went to an international conference on numerical methods
in Haiti. During the conference, an excursion was arranged to a rural area,
and a strange tomato salad consisting of, among other things, pufferfish
venom, was served [[davis2010serpent]](#davis2010serpent).
After arriving back in Norway, both scientists got a fever, stopped eating,
and subsequently started to behave suspiciously.

Dirdal has about 683 inhabitants, and Sokndal 3305 [[snl]](#snl).
We want to use the SZ-model to investigate a potential zombie invasion in
these two places.
For the initial phase of the zombie outbreak, suppose that the two scientists
are staying with their closest family ($S$=2), that the duration of contact is
10 hours, and that there is a 20$\%$ chance of transmitting the infection to
the family members. From this, we estimate a value for $\beta$ as follows:

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
\beta\frac{Z}{N} \Delta{t}=\beta\frac{1}{3}\cdot{10}\text{ hours}=0.20\,,
\label{_auto2} \tag{6}
\end{equation}
$$

from which we find $\beta=0.06$ $\mathrm{\frac{1}{\text{hr}}}$.



**Part 1.**
Make a figure in which you compare the fraction of the total population that
are predicted to turn into zombies in the two locations, using the analytical
solution in equation ([4](#eq:SZ_analytical2)).


**Part 2.**
Explain why, in the simple model described by equations ([2](#eq:SIR1)) and
([3](#eq:SIR2)), the entire human population will be eradicated and
turned into zombies in the limit $t\rightarrow{\infty}$, regardless of the
value of $\beta>0$!

Hint: Rewrite equation ([3](#eq:SIR2)), using that $S(t)=N-Z(t)$.


# Exercise 3: Numerical solver implementation
The SZ-model can be recast into the following general form:

<!-- Equation labels as ordinary links -->
<div id="eq:p3:ode"></div>

$$
\begin{equation}
\label{eq:p3:ode} \tag{7}
\frac{d\mathbf{y}}{dt}=\mathbf{f}(\mathbf{y},t)\,,
\end{equation}
$$

where

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}
\mathbf{y}=(S,Z) 
\label{_auto3} \tag{8}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}  
\mathbf{f}(\mathbf{y},t)=(-\beta S(t)Z(t),\beta S(t)Z(t))\,.
\label{_auto4} \tag{9}
\end{equation}
$$

Develop your own numerical solver(s) that can handle any system of ODEs of
the form ([7](#eq:p3:ode)). This means that the right-hand side
$\mathbf{f}(\mathbf{y},t)$ should be provided as an input argument to a
suitably defined Python function/class. Moreover:
* Implement as one option the Forward Euler Scheme

* Implement as another option the Runge-Kutta fourth order scheme (see [[hiorth2019]](#hiorth2019) and lecture notes)

* Use your implementation to solve the SZ-model numerically. Test your code against the analytical solution, assuming $\beta$ to be equal to the value we estimated above.

# Exercise 4: Predicting the fate of Sokndal and Dirdal
<div id="fate_of_dirdal_and_sokndal"></div>

Later on in a zombie outbreak, people become more aware and not so easily
fooled by the hordes of undead; after all, zombies are not known to
be particularly bright. To capture this behavior, the zombie-infection rate is
assumed to decline exponentially:

<!-- Equation labels as ordinary links -->
<div id="eq:p3:betaT"></div>

$$
\begin{equation}
\beta\to\beta(t)=\beta_0 e^{-\lambda t}.
\label{eq:p3:betaT} \tag{10}
\end{equation}
$$

Suppose moreover that, after a specific time $T$, the probability of getting
infected has been reduced to 60$\%$ of its initial value. We can then estimate $\lambda$ from:

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation}
e^{-\lambda T}=0.6,
\label{_auto5} \tag{11}
\end{equation}
$$

from which it follows that

<!-- Equation labels as ordinary links -->
<div id="eq:p3:pT"></div>

$$
\begin{equation}
\lambda=-\frac{1}{T}\ln 0.6.
\label{eq:p3:pT} \tag{12}
\end{equation}
$$

The updated model can be again be written in the form ([7](#eq:p3:ode)),
the only difference now being that $\beta=\beta(t)$ is time-dependent.
We shall consider two cases:

1. Sokndal outbreak: after 48 hours, the probability of being infected by a zombie has dropped to 60$\%$ of its initial value.

2. Dirdal outbreak: people are very tolerant to unorthodox behavior, which is usually a good thing, but here it has the consequence that it takes 72 hours before the probability of zombie infection drops to 60$\%$ of its initial value.

**Part 1.**

1. Estimate $\lambda$ from the provided information about the Sokndal and Dirdal outbreaks.

2. Solve the SZ-model numerically, but this time replace the constant $\beta$-factor with the time-dependent $\beta_0 e^{-\lambda t}$. Roughly how many people survive the zombie outbreak in the two locations now?

# Exercise 5: Adaptive Method and Richardson extrapolation (CAN BE SKIPPED)
("CAN BE SKIPPED" means that you will not fail skipping this exercise, but you will not get full credits)

1. Extend your ODE solver by implementing an adaptive version of the Runge-Kutta fourth order scheme, using Richardson extrapolation.

2. Check that the adaptive solver produces the same results as in Part 1.

**<font color="white">Speeding up the numerical solver</font>.**

<font color="white">The numerical solver is fast already, but there is still room for
improvement. Who knows, maybe this is exactly the amount of time you need to
escape from the zombie apocalypse...</font>




# Exercise 6: Counter attacks - Kill them all?
<div id="SIR_model"></div>
In our second model the humans might survive, but that is only because the
probability of being infected eventually drops to zero; the zombies are still
wandering around. Let us now refine our model further:
* So far, we have assumed that bitten humans turn into zombies instantaneously. This is a conservative assumption, and seem to be supported by various movies and TV shows. However, other sources [[brooks2003zombie]](#brooks2003zombie) report that a latency / incubation period of about 24 hours may be realistic.

* At any given time, we assume a certain number of zombies to be killed by the humans; they are thus effectively removed from both the human and the zombie populations.

* To make it more realistic, we also allow the humans to perform violent attacks at regular time intervals [[langtangen2013escaping]](#langtangen2013escaping)

<!-- dom:FIGURE: [fig-project3/seir.png, width=400 frac=1.0] SEZR model. <div id="fig:p3:seir"></div> -->
<!-- begin figure -->
<div id="fig:p3:seir"></div>

<p>SEZR model.</p>
<img src="fig-project3/seir.png" width=400>

<!-- end figure -->


Translated to the language of mathematics, we now wish to solve a system
with *four* compartments, see [figure](#fig:p3:seir)

<!-- Equation labels as ordinary links -->
<div id="eq:SIZR_model1"></div>

$$
\begin{equation}
\label{eq:SIZR_model1} \tag{13}
\frac{\mathrm{d}S(t)}{\mathrm{d}t}=-\beta(t)\cdot\frac{S(t)Z(t)}{N}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:SIZR_model2"></div>

$$
\begin{equation}  
\label{eq:SIZR_model2} \tag{14}
\frac{\mathrm{d}E(t)}{\mathrm{d}t}=\beta(t)\cdot\frac{S(t)Z(t)}{N}-\sigma\cdot{E(t)}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:SIZR_model3"></div>

$$
\begin{equation}  
\label{eq:SIZR_model3} \tag{15}
\frac{\mathrm{d}Z(t)}{\mathrm{d}t}=\sigma\cdot{E(t)}-(\alpha+\omega(t))\frac{S(t)Z(t)}{N}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:SIZR_model4"></div>

$$
\begin{equation}  
\label{eq:SIZR_model4} \tag{16}
\frac{\mathrm{d}R(t)}{\mathrm{d}t}=(\alpha+\omega(t))\frac{S(t)Z(t)}{N}\,.
\end{equation}
$$

The first new compartment, $E(t)$, represents the class of exposed humans,
i.e., those that have been bitten or scratched by a zombie, but which still
remain non-contagious to others.
The parameter $\sigma$ is the fraction of such people that turn into zombies
per unit time; alternatively, we may view $1/\sigma$ as the
*incubation time* of the zombie disease, and set it equal to $1/24$ hrs.

The second new compartment, $R(t)$ represents killed (removed) zombies.
In the same way that a certain fraction of human-zombie encounters result in
transmission of the disease, a certain number of encounters also lead to the
human killing the zombie; in the model, the sum $\alpha+\omega(t)$ is therefore
a measure of the probability that the zombie will have its brain
destroyed (or separated from its head).
Alternatively, $1/(\alpha+\omega(t))$ may be viewed as the 'mean infectious
period' of the zombie disease. Note that $\alpha$ is a constant factor, and
may be thought of as the background rate of zombie-killing.
On the other hand, the function
$\omega=\omega(t)$ describe the violent attacks [[langtangen2013escaping]](#langtangen2013escaping).

In [2]:
#attacks occur at specified times following the zombie outbreak:
attacks=[100,124,148,172,196]  # hrs
a=20*beta
def omega(t,a,attacks):
    return a*np.sum([np.exp(-.5*(t-ti)**2) for ti in T])

In [figure](#fig:p3:wt), the function $\omega(t)$ is shown.
Changing the factor in front of `(t-ti)**2` changes the duration of the
attacks. The parameter `a` should be chosen to be much larger than $\beta$,
because humans are much smarter than the zombies, and have more advanced
weapons.

<!-- dom:FIGURE: [fig-project3/wt.png, width=400 frac=1.0] Attack function. <div id="fig:p3:wt"></div> -->
<!-- begin figure -->
<div id="fig:p3:wt"></div>

<p>Attack function.</p>
<img src="fig-project3/wt.png" width=400>

<!-- end figure -->




**Part 1.**
The *basic reproduction number*, $\mathcal{R}_0$, can be thought of as a
number which determine whether a disease can spread through the
population (*very* important for *ebola, HIV, polio, etc*).
It represents the average number of new infected cases produced, in an entirely
susceptible population, by a typical infected
individual [[diekmann1990definition]](#diekmann1990definition).
In our model, it is a good approximation to define it as

<!-- Equation labels as ordinary links -->
<div id="eq:r0_crit"></div>

$$
\begin{equation}
\label{eq:r0_crit} \tag{17}
\mathcal{R}_0\simeq\frac{\beta(t)}{\alpha + \omega(t)}.
\end{equation}
$$

Thus, if $\mathcal{R}_0>1$ we have a zombie apocalypse.


**Part 2.**

* Run some simulations (choose Sokndal or Dirdal), first with $\omega(t)=0$, and with $\beta$ constant. Is it possible to survive the zombie apocalypse if $\beta/\alpha >1$?

* Turn on violent attacks. Is it now possible to survive if $\beta/\alpha>1$?

* For the various scenarios that you consider, what will be the happen to the human community in Dirdal / Sokndal? Support your conclusions with one or more figures, and/or with mathematical arguments.

**<font color="white">if $Z(t)<1$ and $E(t)<1$</font>.**

<font color="white">If the number of zombies or infected drops below one, it makes
sense to set $\beta$ and $\sigma$ $\to0$. There is no such thing as a
fractional zombie or human.</font>




# Exercise 7: Ebola epidemic in West Africa
<div id="ebola"></div>

Finally, we want to adapt our model to handle the case of the Ebola virus.
Data for Ebola outbreaks are freely available at the website of the
[World Health Organization](https://www.who.int) (WHO), and a subset of
the data have been stored in a more convenient format in the following text
files:
* `ebola_cases_guinea.dat`

* `ebola_cases_liberia.dat`

* `ebola_cases_sierra_leone.dat`

In each of these files, the third column contains reported new
cases (confirmed+probable) of the Ebola virus in the given country.
For completeness and to avoid confusion, we present the equations to be
used here:

<!-- Equation labels as ordinary links -->
<div id="eq:SIZRe_model1"></div>

$$
\begin{equation}
\label{eq:SIZRe_model1} \tag{18}
\frac{\mathrm{d}S(t)}{\mathrm{d}t}=-\beta(t)\cdot\frac{S(t)Z(t)}{N}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:SIZRe_model2"></div>

$$
\begin{equation}  
\label{eq:SIZRe_model2} \tag{19}
\frac{\mathrm{d}E(t)}{\mathrm{d}t}=\beta(t)\cdot\frac{S(t)Z(t)}{N}-\sigma\cdot{E(t)}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:SIZRe_model3"></div>

$$
\begin{equation}  
\label{eq:SIZRe_model3} \tag{20}
\frac{\mathrm{d}Z(t)}{\mathrm{d}t}=\sigma\cdot{E(t)}-\gamma Z(t)
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:SIZRe_model4"></div>

$$
\begin{equation}  
\label{eq:SIZRe_model4} \tag{21}
\frac{\mathrm{d}R(t)}{\mathrm{d}t}=\gamma Z(t).
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation}  
\label{_auto6} \tag{22}
\end{equation}
$$

These equations are very similar to equations ([13](#eq:SIZR_model1))-([16](#eq:SIZR_model4)).
However, the compartment $Z$ now represents people infected
with the Ebola virus. Also, $R$ represents previously infected
people that have either 1) recovered (become immune), or 2) died.
For simplicity, we do not distinguish between these two above alternatives;
persons in both categories are assumed to not have any bearing on whether
the virus will continue to spread.
Accordingly, we model the rate of 'removal' from the Z-compartment with a
term $\gamma Z$, in which the constant $1/\gamma$ is the average time until
recovery/death (mean infectious period). As done previously, we compute
$\beta$ from

<!-- Equation labels as ordinary links -->
<div id="_auto7"></div>

$$
\begin{equation}
\beta(t)=\beta_0e^{-\lambda t},
\label{_auto7} \tag{23}
\end{equation}
$$

where $\beta_0$ and $\lambda$ are *unknown* parameters; they need to be
determined from data.


**Part 1.**
For each of the three West African countries, make a figure in which you
plot

1. the number of new cases of Ebola registered at any given moment in time, and

2. the cumulative amount.

If done correctly, the end result for Guinea should look something
like [figure](#fig:ebola_guinea).

<!-- dom:FIGURE: [fig-project3/ebola_guinea.png, width=400 frac=1.0] Outbreak of the Ebola virus in Guinea, 2014-2015. <div id="fig:ebola_guinea"></div> -->
<!-- begin figure -->
<div id="fig:ebola_guinea"></div>

<p>Outbreak of the Ebola virus in Guinea, 2014-2015.</p>
<img src="fig-project3/ebola_guinea.png" width=400>

<!-- end figure -->



**Part 2.**
It has been estimated that the average infectious period for
the Ebola virus is one week, and that the latency period is
9.7 days [[wong2017systematic]](#wong2017systematic), hence $1/\gamma=7$ days, and $1/\sigma=9.7$ days.
Assuming these parameters to be valid, we want to fit the four-compartment model
with an exponentially decaying $\beta$-factor to the data.
For simplicity, we will assume a fixed population size of $N=10^7$ in all
three countries; the exact value should not be that important as long as
$S(t)\approx{N}$.

For the case of Guinea, experiment manually by varying $\beta_0$ and $\lambda$
until the model produces something that resembles
[figure](#fig:ebola_guinea).


**Part 3.**
Next, we want to use an automatic optimization method to fit the model to the
data for all three countries. One possibility
is to use [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.curve_fit.html).
You may for example take the following Python code as a starting point:

In [3]:
# To do: store time in days in array t
# To do: store cumulative number of Ebola cases in array Ct

def non_linear_func(t, beta0, lam):
    # To do:
    # Solve system of ODEs and return cumulative number of
    # outbreaks at each time t.
    pass
bds = ((0.0, 0.0), (np.inf, np.inf))
popt, pcov = sp.optimize.curve_fit(non_linear_func, t, Ct, bounds=bds)
print('Curve-fit: beta0={}, lambda={}.'.format(popt[0], popt[1]))

For each country, make a plot like [figure](#fig:ebola_guinea) in which
you compare the model to the data.

**<font color="white">Note:</font>.**

<font color="white">For Liberia, the number of reported outbreaks are very sporadic initially.
Therefore, you might need to start counting days at a later date than what
is done in the text file.</font>




**Part 4.**
Discuss the results from the previous exercise. How does the basic
reproductive number
$\mathcal{R}_0\simeq\frac{\beta(t)}{\gamma}$ differ among the various
countries, and how does that correlate with the spreading of the disease?
How do you think these kinds of models can be useful in practice?


**Part 5. (OPTIONAL)**
Suggest one or more further modifications to the model. Investigate your
updated model by varying the input parameters and plotting the results.
If possible, you might also find a more comprehensive data set to compare
your model with.

# Guidelines for project submission
The assignment is provided both as a PDF, and as a Jupyter notebook.
However, the work done to answer the exercises only has to be handed in as a
notebook, though you can submit an additional PDF if you want.
You should bear the following points in mind when working on the project:
* Start your notebook by providing a short introduction in which you outline the nature of the problem(s) to be investigated.

* End your notebook with a brief summary of what you feel you learned from the project (if anything). Also, if you have any general comments or suggestions for what could be improved in future assignments, this is the place to do it.

* All code that you make use of should be present in the notebook, and it should ideally execute without any errors (especially run-time errors). If you are not able to fix everything before the deadline, you should give your best understanding of what is not working, and how you might go about fixing it.

* If you use an algorithm that is not fully described in the assignment text, you should try to explain it in your own words. This also applies if the method is described elsewhere in the course material.

* In some cases it may suffice to explain your work via comments in the code itself, but other times you might want to include a more elaborate explanation in terms of, e.g., mathematics and/or pseudocode.

* In general, it is a good habit to comment your code (though it can be overdone).

* When working with approximate solutions to equations, it is always useful to check your results against known exact (analytical) solutions, should they be available.

* It is also a good test of a model implementation to study what happens at known 'edge cases'.

* Any figures you include should be easily understandable. You should label axes appropriately, and depending on the problem, include other legends etc. Also, you should discuss your figures in the main text.

* It is always good if you can reflect a little bit around *why* you see what you see.

# Bibliography
1. <div id="kermack1927contribution"></div> **W. O. Kermack and A. G. McKendrick**. 
    A Contribution to the Mathematical Theory of Epidemics,
    *Proceedings of the royal society of london. Series A, Containing papers of a mathematical and physical character*,
    115(772),
    pp. 700-721,
    1927.

2. <div id="begon2002clarification"></div> **M. Begon, M. Bennett, R. G. Bowers, N. P. French, S. Hazel and J. Turner**. 
    A Clarification of Transmission Terms in Host-Microparasite Models: Numbers, Densities and Areas,
    *Epidemiology & Infection*,
    129(1),
    pp. 147-153,
    2002.

3. <div id="davis2010serpent"></div> **W. Davis**. 
    *The Serpent and the Rainbow*,
    New York: Simon and Schuster,
    2010.

4. <div id="snl"></div> Store Norske Leksikon. 
    Accessed: 2019-10-24,
    <https://snl.no>.

5. <div id="hiorth2019"></div> **A. Hiorth**. 
    *Computational Engineering and Modeling*,
    https://github.com/ahiorth/CompEngineering,
    2019.

6. <div id="brooks2003zombie"></div> **M. Brooks**. 
    *The Zombie Survival Guide: Complete Protection From the Living Dead*,
    Broadway books,
    2003.

7. <div id="langtangen2013escaping"></div> **H. P. Langtangen, K.-A. Mardal and P. Røtnes**. 
    Escaping the Zombie Threat by Mathematics,
    *Zombies in the Academy-Living Death in Higher Education*,
    2013.

8. <div id="diekmann1990definition"></div> **O. Diekmann, J. A. P. Heesterbeek and J. A. Metz**. 
    On the Definition and the Computation of the Basic Reproduction Ratio R 0 in Models for Infectious Diseases in Heterogeneous Populations,
    *Journal of mathematical biology*,
    28(4),
    pp. 365-382,
    1990.

9. <div id="wong2017systematic"></div> **Z. Wong, C. Bui, A. Chughtai and C. Macintyre**. 
    A Systematic Review of Early Modelling Studies of Ebola Virus Disease in West Africa,
    *Epidemiology & Infection*,
    145(6),
    pp. 1069-1094,
    2017.